In [ ]:
import tensorflow.keras.models
import numpy as np
import tensorflow as tf
import os
import cv2
import pandas as pd

path_to_model = "C:\\Users\\kseni\\JUPYTER\\DM-STAVROPOL\\models\\NN_acc_50"

model = tf.keras.models.load_model(path_to_model)

dicti = {0:"cartwheel",
1:"catch",
2:"clap",
3:"climb",
4:"dive",
5:"draw_sword",
6:"dribble",
7:"fencing",
8:"flic_flac",
9:"golf",
10:"handstand",
11:"hit",
12:"jump",
13:"pick",
14:"pour",
15:"pullup",
16:'push',
17:"pushup",
18:"shoot_ball",
19:"sit",
20:"situp",
21:"swing_baseball",
22:"sword_exercise",
23:"throw"
}

count_cadrs = 28
x_size = 144
y_size = 144
count_classes = 23
n_ports = 3
n_videos = 4
test_dataset_path = "C:\\Users\\kseni\\JUPYTER\\DM-STAVROPOL\\test_dataset"


def getFrames(video_paths):
    data_frame = pd.DataFrame(columns=['video'])
    index = 0
    url = 'C:\\Users\\kseni\\JUPYTER\\DM-STAVROPOL\\test_dataset'
    list_names = os.listdir(url)

    for name in list_names:
        print(name)
        new_url_1 = url+f'\\{name}'
        cap = cv2.VideoCapture(new_url_1)
        num = -1
        frames = []

        while(cap.isOpened()):
            num += 1
            if num in [2, 4]:
                continue
            ret, frame = cap.read()
            if (ret == True):
                if num % 2 == 0:
                    # Преобразование кадра в тензор
                    frame = cv2.resize(frame, (x_size, y_size)) # Изменение размера кадра
                    frame = frame / 255.0 # Нормализация пикселей
                    frame = np.expand_dims(frame, axis=[0]) # Добавление измерения для тензора
                    frames.append(frame)
            else:
                break
            if len(frames) == count_cadrs:
                #print('Попалось видео с недостатком кадров')
                break

        # Преобразование списка тензоров в тензор
        try:
            frames = np.concatenate(frames, axis=0)
        except ValueError:
            pass
        if frames.shape[0] < count_cadrs:
            continue    
        #print(frames.shape)
        data_frame.loc[index, 'video'] = frames
        index += 1
    X = np.array(data_frame['video'].tolist())
    return X



def predict(path_to_video):
    return np.argmax(model.predict(getFrames(path_to_video)))

answers = predict(os.listdir(test_dataset_path))
print(answers)